Deep Learning
=============

Assignment 2
------------

Previously in `1_notmnist.ipynb`, we created a pickle with formatted datasets for training, development and testing on the [notMNIST dataset](http://yaroslavvb.blogspot.com/2011/09/notmnist-dataset.html).

The goal of this assignment is to progressively train deeper and more accurate models using TensorFlow.

In [1]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

First reload the data we generated in `1_notmnist.ipynb`.

In [2]:
pickle_file = 'notMNIST.pickle'

with open(pickle_file, 'rb') as f:
  save = pickle.load(f)
  train_dataset = save['train_dataset']
  train_labels = save['train_labels']
  valid_dataset = save['valid_dataset']
  valid_labels = save['valid_labels']
  test_dataset = save['test_dataset']
  test_labels = save['test_labels']
  del save  # hint to help gc free up memory
  print('Training set', train_dataset.shape, train_labels.shape)
  print('Validation set', valid_dataset.shape, valid_labels.shape)
  print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 28, 28) (200000,)
Validation set (10000, 28, 28) (10000,)
Test set (10000, 28, 28) (10000,)


Reformat into a shape that's more adapted to the models we're going to train:
- data as a flat matrix,
- labels as float 1-hot encodings.

In [3]:
image_size = 28
num_labels = 10

def reformat(dataset, labels):
  dataset = dataset.reshape((-1, image_size * image_size)).astype(np.float32)
  # Map 0 to [1.0, 0.0, 0.0 ...], 1 to [0.0, 1.0, 0.0 ...]
  labels = (np.arange(num_labels) == labels[:,None]).astype(np.float32)
  return dataset, labels
train_dataset, train_labels = reformat(train_dataset, train_labels)
valid_dataset, valid_labels = reformat(valid_dataset, valid_labels)
test_dataset, test_labels = reformat(test_dataset, test_labels)
print('Training set', train_dataset.shape, train_labels.shape)
print('Validation set', valid_dataset.shape, valid_labels.shape)
print('Test set', test_dataset.shape, test_labels.shape)

Training set (200000, 784) (200000, 10)
Validation set (10000, 784) (10000, 10)
Test set (10000, 784) (10000, 10)


We're first going to train a multinomial logistic regression using simple gradient descent.

TensorFlow works like this:
* First you describe the computation that you want to see performed: what the inputs, the variables, and the operations look like. These get created as nodes over a computation graph. This description is all contained within the block below:

      with graph.as_default():
          ...

* Then you can run the operations on this graph as many times as you want by calling `session.run()`, providing it outputs to fetch from the graph that get returned. This runtime operation is all contained in the block below:

      with tf.Session(graph=graph) as session:
          ...

Let's load all the data into TensorFlow and build the computation graph corresponding to our training:

In [4]:
# With gradient descent training, even this much data is prohibitive.
# Subset the training data for faster turnaround.
train_subset = 10000

graph = tf.Graph()
with graph.as_default():

  # Input data.
  # Load the training, validation and test data into constants that are
  # attached to the graph.
  tf_train_dataset = tf.constant(train_dataset[:train_subset, :])
  tf_train_labels = tf.constant(train_labels[:train_subset])
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  # Variables.
  # These are the parameters that we are going to be training. The weight
  # matrix will be initialized using random valued following a (truncated)
  # normal distribution. The biases get initialized to zero.
  weights = tf.Variable(
    tf.truncated_normal([image_size * image_size, num_labels]))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  # We multiply the inputs with the weight matrix, and add biases. We compute
  # the softmax and cross-entropy (it's one operation in TensorFlow, because
  # it's very common, and it can be optimized). We take the average of this
  # cross-entropy across all training examples: that's our loss.
  logits = tf.matmul(tf_train_dataset, weights) + biases
  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
  
  # Optimizer.
  # We are going to find the minimum of this loss using gradient descent.
  optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  
  # Predictions for the training, validation, and test data.
  # These are not part of training, but merely here so that we can report
  # accuracy figures as we train.
  train_prediction = tf.nn.softmax(logits)
  valid_prediction = tf.nn.softmax(
    tf.matmul(tf_valid_dataset, weights) + biases)
  test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)

Let's run this computation and iterate:

In [5]:
num_steps = 801

def accuracy(predictions, labels):
  return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1))
          / predictions.shape[0])

with tf.Session(graph=graph) as session:
  # This is a one-time operation which ensures the parameters get initialized as
  # we described in the graph: random weights for the matrix, zeros for the
  # biases. 
  tf.initialize_all_variables().run()
  print('Initialized')
  for step in range(num_steps):
    # Run the computations. We tell .run() that we want to run the optimizer,
    # and get the loss value and the training predictions returned as numpy
    # arrays.
    _, l, predictions = session.run([optimizer, loss, train_prediction])
    if (step % 100 == 0):
      print('Loss at step %d: %f' % (step, l))
      print('Training accuracy: %.1f%%' % accuracy(
        predictions, train_labels[:train_subset, :]))
      # Calling .eval() on valid_prediction is basically like calling run(), but
      # just to get that one numpy array. Note that it recomputes all its graph
      # dependencies.
      print('Validation accuracy: %.1f%%' % accuracy(
        valid_prediction.eval(), valid_labels))
  print('Test accuracy: %.1f%%' % accuracy(test_prediction.eval(), test_labels))

Instructions for updating:
Use `tf.global_variables_initializer` instead.
Initialized
Loss at step 0: 16.887674
Training accuracy: 9.1%
Validation accuracy: 12.1%
Loss at step 100: 2.261385
Training accuracy: 72.3%
Validation accuracy: 71.4%
Loss at step 200: 1.834082
Training accuracy: 75.0%
Validation accuracy: 73.5%
Loss at step 300: 1.595344
Training accuracy: 76.1%
Validation accuracy: 74.0%
Loss at step 400: 1.434985
Training accuracy: 77.1%
Validation accuracy: 74.5%
Loss at step 500: 1.318047
Training accuracy: 77.8%
Validation accuracy: 74.8%
Loss at step 600: 1.227451
Training accuracy: 78.4%
Validation accuracy: 74.8%
Loss at step 700: 1.154096
Training accuracy: 78.9%
Validation accuracy: 75.0%
Loss at step 800: 1.092825
Training accuracy: 79.4%
Validation accuracy: 75.2%
Test accuracy: 82.6%


Let's now switch to stochastic gradient descent training instead, which is much faster.

The graph will be similar, except that instead of holding all the training data into a constant node, we create a `Placeholder` node which will be fed actual data at every call of `session.run()`.

A dual layer 1024 ReLu network
------------------------------

For the highest validation (1024/1024) we get:

Cost=0.001. For highest validation accuracy: 90.1%, I got a test accuracy: 95.7%

Cost=0.0005. For highest validation accuracy: 90.6%, I got a test accuracy: 95.9%

Cost=0.0001. For highest validation accuracy: 91.0%, I got a test accuracy: 95.9% (but get NaN's at some point)

Cost=0. For highest validation accuracy: 91.0%, I got a test accuracy: 96.1%

Slightly better with 2048/1024:

Cost=0. For highest validation accuracy: 91.2%, I got a test accuracy: 96.4%



In [6]:
import math

batch_size = 128*4

def variable_summaries(var):
  """Attach a lot of summaries to a Tensor (for TensorBoard visualization)."""
  with tf.name_scope('summaries'):
    mean = tf.reduce_mean(var)
    tf.summary.scalar('mean', mean)
    tf.summary.scalar('absmean', tf.reduce_mean(tf.abs(var)))
    
    with tf.name_scope('stddev'):
      stddev = tf.sqrt(tf.reduce_mean(tf.square(var - mean)))
    tf.summary.scalar('stddev', stddev)
    tf.summary.scalar('max', tf.reduce_max(var))
    tf.summary.scalar('min', tf.reduce_min(var))
    tf.summary.histogram('histogram', var)

graph = tf.Graph()
with graph.as_default():

  # Input data. For the training data, we use a placeholder that will be fed
  # at run time with a training minibatch.
  tf_train_dataset = tf.placeholder(tf.float32,
                                    shape=(None, image_size * image_size))
  tf_train_labels = tf.placeholder(tf.float32, shape=(None, num_labels))
  tf_valid_dataset = tf.constant(valid_dataset)
  tf_test_dataset = tf.constant(test_dataset)
  
  with tf.name_scope('hl_1_weights'):
    hidden0 = 1024
    weights0 = tf.Variable(
      tf.truncated_normal([image_size * image_size, hidden0],stddev=math.sqrt(1.0/(image_size * image_size))))
    biases0 = tf.Variable(tf.zeros([hidden0]))
    variable_summaries(weights0)
    
  with tf.name_scope('hl_2_weights'):
    hidden1 = 1024
    weights1 = tf.Variable(
      tf.truncated_normal([hidden0, hidden1],stddev=math.sqrt(1.0/hidden0)))
    biases1 = tf.Variable(tf.zeros([hidden1]))
    variable_summaries(weights1)
    

  # Variables.
  weights = tf.Variable(tf.truncated_normal([hidden1, num_labels],stddev=math.sqrt(1.0/hidden1)))
  biases = tf.Variable(tf.zeros([num_labels]))
  
  # Training computation.
  out_layer_1 = tf.matmul(tf_train_dataset, weights0) + biases0
  out_layer_1_relu = tf.nn.relu(out_layer_1)

  out_layer_2 = tf.matmul(out_layer_1_relu, weights1) + biases1
  out_layer_2_relu = tf.nn.relu(out_layer_2)

  #keep_prob = tf.placeholder(tf.float32)
  #relu_drop = tf.nn.dropout(relu, keep_prob)

  logits = tf.matmul(out_layer_2_relu, weights) + biases

  lossC = 0.0001*0

  loss = tf.reduce_mean(
    tf.nn.softmax_cross_entropy_with_logits(labels=tf_train_labels, logits=logits) + lossC*tf.nn.l2_loss(weights) +
        lossC*tf.nn.l2_loss(weights0) + lossC*tf.nn.l2_loss(weights1))
  
    
    
  # Optimizer.
  #optimizer = tf.train.GradientDescentOptimizer(0.5).minimize(loss)
  optimizer = tf.train.MomentumOptimizer(0.5,0.1).minimize(loss)
  #optimizer = tf.train.AdadeltaOptimizer().minimize(loss)
  #optimizer = tf.train.AdamOptimizer().minimize(loss)
     

    
  # Predictions for the training, validation, and test data.
  train_prediction = tf.nn.softmax(logits)
    
    
  #valid_prediction = tf.nn.softmax(tf.matmul(tf_valid_dataset, weights) + biases)
  #test_prediction = tf.nn.softmax(tf.matmul(tf_test_dataset, weights) + biases)
  
  #MHC: This one was tricky - in order to log accuracy it must be calculated in the graph, and not using the 'accuracy' method.
  correct_prediction = tf.equal(tf.argmax(train_prediction,1), tf.argmax(tf_train_labels,1))
  myAccuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
  

Let's run it:

In [7]:
num_steps = 15002
import time
start = time.time()

dir = "D:\TensorflowLogs"
train_writer = tf.summary.FileWriter(dir + '/train', session.graph)
test_writer = tf.summary.FileWriter(dir + '/test')
validate_writer = tf.summary.FileWriter(dir + '/validate')

#MHC: To start Tensorboard (Windows): 'Tensorboard --logdir=d:d:\TensorflowLogs7'


with tf.Session(graph=graph) as session:
  tf.global_variables_initializer().run()
  tf.summary.scalar('loss', loss)
  tf.summary.scalar('error (1 minus accuracy)', 1.0-myAccuracy)
  merged = tf.summary.merge_all()

  print("Initialized")
  for step in range(num_steps):
    # Pick an offset within the training data, which has been randomized.
    # Note: we could use better randomization across epochs.
    offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
    # Generate a minibatch.
    batch_data = train_dataset[offset:(offset + batch_size), :]
    batch_labels = train_labels[offset:(offset + batch_size), :]
    # Prepare a dictionary telling the session where to feed the minibatch.
    # The key of the dictionary is the placeholder node of the graph to be fed,
    # and the value is the numpy array to feed to it.
    feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
    _, l  = session.run([optimizer, loss], feed_dict=feed_dict)
    if (step%100==99):
      print("Minibatch loss at step %d: %f (%fs)" % (step, l, time.time()-start))
      feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
      summary, predictions = session.run([merged,train_prediction], feed_dict=feed_dict)
      print("Train accuracy: %.1f%%" % accuracy(predictions, batch_labels))
      train_writer.add_summary(summary, step)
    
      feed_dict = {tf_train_dataset : valid_dataset, tf_train_labels : valid_labels}
      summary, predictions = session.run([merged,train_prediction], feed_dict=feed_dict)
      print("Validation accuracy: %.1f%%" % accuracy(predictions, valid_labels))
      validate_writer.add_summary(summary, step)
    
      feed_dict = {tf_train_dataset : test_dataset, tf_train_labels : test_labels}
      summary, predictions = session.run([merged,train_prediction], feed_dict=feed_dict)
      print("Test accuracy: %.1f%%" % accuracy(predictions, test_labels))
      test_writer.add_summary(summary, step)
    

INFO:tensorflow:Summary name error (1 minus accuracy) is illegal; using error__1_minus_accuracy_ instead.
Initialized
Minibatch loss at step 99: 0.568825 (23.394306s)
Train accuracy: 86.1%
Validation accuracy: 84.0%
Test accuracy: 90.4%
Minibatch loss at step 199: 0.481263 (26.985020s)
Train accuracy: 88.1%
Validation accuracy: 85.4%
Test accuracy: 92.5%
Minibatch loss at step 299: 0.444236 (30.055810s)
Train accuracy: 88.5%
Validation accuracy: 85.8%
Test accuracy: 92.5%
Minibatch loss at step 399: 0.399999 (33.131241s)
Train accuracy: 90.6%
Validation accuracy: 86.9%
Test accuracy: 93.5%
Minibatch loss at step 499: 0.383614 (36.114260s)
Train accuracy: 92.8%
Validation accuracy: 87.8%
Test accuracy: 93.9%
Minibatch loss at step 599: 0.379923 (39.072215s)
Train accuracy: 92.6%
Validation accuracy: 87.1%
Test accuracy: 93.3%
Minibatch loss at step 699: 0.324794 (42.064555s)
Train accuracy: 93.6%
Validation accuracy: 88.5%
Test accuracy: 94.5%
Minibatch loss at step 799: 0.372165 (45.05

---
Problem
-------

Turn the logistic regression example with SGD into a 1-hidden layer neural network with rectified linear units [nn.relu()](https://www.tensorflow.org/versions/r0.7/api_docs/python/nn.html#relu) and 1024 hidden nodes. This model should improve your validation / test accuracy.

---